In [1]:
import pandas as pd
import seaborn as sns

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [73]:
import math

In [2]:
#!pip install pyarrow

In [3]:
df_jan = pd.read_parquet('C:/Users/monal/Downloads/yellow_tripdata_2022-01.parquet')

In [11]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [53]:
df_feb = pd.read_parquet('C:/Users/monal/Downloads/yellow_tripdata_2022-02.parquet')

In [12]:
df_feb.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-02-01 00:06:58,2022-02-01 00:19:24,1.0,5.40,1.0,N,138,252,1,17.0,1.75,0.5,3.90,0.00,0.3,23.45,0.0,1.25
1,1,2022-02-01 00:38:22,2022-02-01 00:55:55,1.0,6.40,1.0,N,138,41,2,21.0,1.75,0.5,0.00,6.55,0.3,30.10,0.0,1.25
2,1,2022-02-01 00:03:20,2022-02-01 00:26:59,1.0,12.50,1.0,N,138,200,2,35.5,1.75,0.5,0.00,6.55,0.3,44.60,0.0,1.25
3,2,2022-02-01 00:08:00,2022-02-01 00:28:05,1.0,9.88,1.0,N,239,200,2,28.0,0.50,0.5,0.00,3.00,0.3,34.80,2.5,0.00
4,2,2022-02-01 00:06:48,2022-02-01 00:33:07,1.0,12.16,1.0,N,138,125,1,35.5,0.50,0.5,8.11,0.00,0.3,48.66,2.5,1.25


# Number of columns in Jan 2022 Yellow Taxi Trip data

In [4]:
len(df_jan.columns)

19

# Standard deviation of the trips duration in Jan 2022 Yellow Taxi Trip data

In [5]:
(df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime).std().seconds / 60

46.43333333333333

# Dropping outliers

In [6]:
len(df_jan.tpep_dropoff_datetime)

2463931

In [55]:
df_duration = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime

In [56]:
df_duration = df_duration.dt.total_seconds()

In [57]:
df_duration_feb = df_feb.tpep_dropoff_datetime - df_feb.tpep_pickup_datetime

In [58]:
df_duration_feb = df_duration_feb.dt.total_seconds()

In [9]:
with_outliers = len(df_duration)

In [10]:
lower_limit = 1 * 60
upper_limit = 60 * 60

df_duration_outliers_Removed = [i for i in df_duration if i >= lower_limit and i <= upper_limit]

In [11]:
outliers_rem = len(df_duration_outliers_Removed)

In [12]:
outliers_rem / with_outliers

0.9827547930522406

# Dimensionality after OHE

In [13]:
print(len(df_jan['PULocationID'].unique()))
print(len(df_jan['DOLocationID'].unique()))

257
261


In [14]:
final_df = df_jan[['PULocationID', 'DOLocationID']].copy()

In [59]:
final_df_feb = df_feb[['PULocationID', 'DOLocationID']].copy()

In [15]:
final_df.columns

Index(['PULocationID', 'DOLocationID'], dtype='object')

In [43]:
one_hot_encoded_data = pd.get_dummies(final_df, columns = ['PULocationID', 'DOLocationID'])
print(len(one_hot_encoded_data.columns))

518


In [60]:
one_hot_encoded_data_feb = pd.get_dummies(final_df_feb, columns = ['PULocationID', 'DOLocationID'])
print(len(one_hot_encoded_data_feb.columns))

516


# Turning the dataframe into a list of dictionaries
having memory issue so i have selected till 70639 rows

In [44]:
one_hot_encoded_data = one_hot_encoded_data[0:70639].to_dict('records')

In [61]:
one_hot_encoded_data_feb = one_hot_encoded_data_feb[0:70639].to_dict('records')

# Fit a dictionary vectorizer


In [45]:
dict_vectorize = DictVectorizer(sparse=False)

# Get a feature matrix from it

In [46]:
X_train = dict_vectorize.fit_transform(one_hot_encoded_data)

In [62]:
X_val = dict_vectorize.transform(one_hot_encoded_data_feb)

In [47]:
X_train.shape

(70639, 518)

In [48]:
y_train = df_duration.values[0:70639] / 60
y_train.shape

(70639,)

In [63]:
y_val = df_duration_feb.values[0:70639] / 60

In [49]:
y_train[0]

17.816666666666666

In [64]:
y_val[0]

12.433333333333334

#  Training a model

In [50]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [76]:
y_pred = lr.predict(X_train)

In [78]:
mse_train = mean_squared_error(y_train, y_pred, squared=False)

In [79]:
RMSE_train = math.sqrt(mse_train)
print(RMSE_train)

8.134281288015298


# Evaluating the model

In [67]:
y_pred_val = lr.predict(X_val)

In [70]:
mse_val = mean_squared_error(y_val, y_pred_val, squared=False)

In [71]:
print(mse_val)

1117418393736.596


In [74]:
RMSE_val = math.sqrt(mse_val)

In [75]:
print(RMSE_val)

1057080.1264504958


In [69]:
print(y_val[0], y_pred_val[0])

12.433333333333334 17.982421875
